# Agents and Tools

In this section of the lab, we will review some agent concepts and see how agents can interact the basic tools

- Define a basic agent, using the `builtin::websearch` tool from Llama Stack
- Get familiar with concepts like "Turns" and "Steps" and "Tool_Calling" that the agent takes
- Run the agent to return web results
- Optional: experience a prompt chaining example

> **Note:**
>This section is intended only as an introduction to Agent concepts and language, and we will get into more exciting work in the next modules. If you are familiar with these concepts, feel free to skip this section.
> 


> **Note:**
> For the sake of simplicity and time, we will skip through some of the python basics we went through in previous modules and treat this more like a nomral python application 
> 

### Install Python Prerequisist

As always, let's start by installing the Python Libraries we neeed, we will add some additional libraries to support some of our tools. 

In [1]:
!pip install -U llama-stack-client==0.2.5 dotenv geocoder

### Define the LLamastack server and Model

Let's define the libraries we need and our basic Llama Stack configurations:

In [2]:
import os

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient

# These libraries are just here to print the results from the agent in a more human-readable way 
from src.utils import step_printer
from termcolor import cprint
import uuid
from llama_stack_client.lib.agents.event_logger import EventLogger
stream=False ## Defaulting to False, you can change this throughout the section to "True" if you wanted to see the output in another format (Using EventLogger)

# for our lab, we will just define our variables manualy here, in a regular application, this would be ready directly from the local .env file and we would comment these lines out
os.environ['LLAMA_STACK_SERVER'] = 'http://localhost:8321'

# We will be using the Tavily web search service (docs.tavily.com/)
tavily_search_api_key='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'
provider_data = {"tavily_search_api_key": tavily_search_api_key}

> **Note:**
>When running this code in a regular Python application, we would usually like to read environment variables from an `.env` file, for our needs in this lab, we will hard code these in this cell, to make things more clear
>

### Initialize the *Client* 
We will not define our client, and introduce a new pattern/trick that will help us later when we try to move between models and other providers within Llama Stack. 

In [3]:
LLAMA_STACK_SERVER=os.getenv("LLAMA_STACK_SERVER")

client = LlamaStackClient(
    base_url=LLAMA_STACK_SERVER,
    provider_data=provider_data
)
# List available models and select from allowed models list
allowed_models_list=["meta-llama/Llama-3.2-3B-Instruct"]
selected_model = None

models = client.models.list()

print("--- Available models: ---")
for m in models:
    print(f"{m.identifier} - {m.provider_id} - {m.provider_resource_id}")
    # Check if the model identifier contains any of the allowed substrings
    if any(substring in m.identifier for substring in allowed_models_list):
        # Only set selected_model if it hasn't been set yet
        if selected_model is None:
            selected_model = m.identifier
           
# If no allowed model was found, you might want to handle that case
if selected_model is None:
    print("No allowed model found in the list.")


print(f"Selected model (from allowed list): {selected_model}")
            # Removed the break here to show all available models, but the selection logic remains picking the first one


SELECTED_MODEL = selected_model

--- Available models: ---
all-MiniLM-L6-v2 - ollama - all-minilm:latest
granite3.2:8b - ollama - granite3.2:8b
meta-llama/Llama-3.2-3B-Instruct - ollama - llama3.2:3b-instruct-fp16
Selected model (from allowed list): meta-llama/Llama-3.2-3B-Instruct


### Define the a single tool agent
Here we define our new agent, we will pass along the `client` defining our connection to the Llama Stack server, the `model` we selected in the previous step, the `tools` array with all available tools for the agent, and crucually, the prompt, named `instructions` for the agent.

> **Note:**
> You might notice that we are being very specific in our prompt, for the purpose of lab we are using rather small models and are providing a narrow path to follow, in a real world scenario, prompts and LLMs can be experimented with to produce the best results.
> 

In [4]:
from llama_stack_client.lib.agents.agent import Agent

agent = Agent(
    client, 
    model=SELECTED_MODEL,
    instructions="""You are a helpful websearch assistant. When you are asked to search the latest you must use a tool. 
            Whenever a tool is called, be sure return the response in a friendly and helpful tone.
            """ ,
    tools=["builtin::websearch"],
)

### inspecting the Agent's response structure

Let's give our agent a couple of easy questions to see how it calls our `builtin::websearch` tool.
Notice that we are generating a new session ID for each question as convesation memory is shared throughtout the session. (More on this later!)


In [11]:
user_prompts = [
       "search the web for the latest in OpenShift?",
       "search for the current weather in boston",
]


for prompt in user_prompts:
    # Generate a new Unique Identifier for each session 
    new_uuid = uuid.uuid4()
    session_id = agent.create_session(f"web-session-{new_uuid}")

    cprint(f"\n{'='*100}\nProcessing user query: {prompt}\n{'='*100}")

    
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: search the web for the latest in OpenShift?

---------- 📍 Step 1: InferenceStep ----------
🤖 Model Response:
<|python_tag|>brave_search(query="latest news OpenShift")

========== Query processing completed ========== 


Processing user query: search for the current weather in boston

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'current weather in Boston'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'current weather in Boston',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in Boston',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Boston', 'region': 'Massachusetts', 'country': 'United States of America', 'lat': 42.3583, 'lon': -71.0603, 'tz_id': 'America/New_York', 'localtime_epoch': 1747338413, 'localtime': '2025-05-15 15:46'}, 'current': {'last_updated_epoch': 1747338300, 'last_updated': '2025-05-15 15:45', 'temp_c': 22.8, 'temp_f': 73.0, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 4.3, 'wind_kph': 6.8, 'wind_degree': 147, 'wind_dir': 'SSE', 'pressure_mb': 1012.0, 'pressure_in': 29.89, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 76, 'cloud': 100, 'feelslike_c': 24.9, 'feelslike_f': 76.9, 'windchill_c': 20.7, 'windchill_f': 69.2, 'heatindex_c': 23.1, 'heatindex_f': 73.5, 'dewpoint_c': 16.7, 'dewpoint_f': 62.1, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 4.0, 'gust_mph': 5.2, 'gust_kph': 8.3}}",
│   │   │   'score': 0.9105978,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Thursday, May 15, 2025. Boston, MA - Weather Forecast',
│   │   │   'url': 'https://weathershogun.com/weather/usa/ma/boston/2111/may/2025-05-15',
│   │   │   'content': 'Boston, Massachusetts weather forecast for Thursday, May 15, 2025. Get the latest on temperature, precipitation, wind speed, and UV. Plan your day with accurate weather updates.',
│   │   │   'score': 0.87348515,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Next Weather: WBZ morning forecast for May 15, 2025 - CBS Boston',
│   │   │   'url': 'https://www.cbsnews.com/boston/video/next-weather-wbz-morning-forecast-for-may-15-2025/',
│   │   │   'content': 'Next Weather: WBZ morning forecast for May 15, 2025 Jason Mikell has your latest weather forecast.Jason Mikell has your latest weather forecast.',
│   │   │   'score': 0.75421005,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Weather in Boston in May 2025 (Massachusetts) - detailed Weather ...',
│   │   │   'url': 'https://world-weather.info/forecast/usa/boston/may-2025/',
│   │   │   'content': "World United States Massachusetts Weather in Boston Weather in Boston in May 2025 Boston Weather Forecast for May 2025 is based on long term prognosis and previous years' statistical data.",
│   │   │   'score': 0.70531887,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Boston weather in May 2025 | Boston 14 day weather',
│   │   │   'url': 'https://www.weather25.com/north-america/usa/massachusetts/boston?page=month&month=May',
│   │   │   'content': 'Full weather forecast for Boston in May 2025. Check the temperatures, chance of rain and more in Boston during May.',
│   │   │   'score': 0.65146625,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
The current weather in Boston is overcast with a temperature of 22.8°C (73.0°F). The wind is blowing at 4.3 mph (6.8 km/h) from the southeast, and the pressure is 1012 mbar. There is no precipitation expected today.

========== Query processing completed ========== 



This output shows an AI agent processing a user query about "latest news OpenShift."

* **📍 Step 1: InferenceStep**: The agent *infers* the need to search for information and decides to use the `brave_search` tool with the query "latest news OpenShift."
* **📍 Step 2: ToolExecutionStep**: The agent *executes* the `brave_search` tool. The JSON block is the *output* from this tool, providing search results with titles, URLs, snippets, and relevance scores.
* **📍 Step 3: InferenceStep**: The agent *processes* the search results from the tool and *infers* how to construct a final answer.
* **🤖 Model Response**: This is the AI agent's *final generated response* to the user's query, summarizing the key information extracted from the search results.

### Creating our first custom tool

This code defines a custom tool called `get_location` for agent to use

* **`@client_tool`**: This decorator from `llama_stack_client.lib.agents.client_tool` marks the `get_location` function as a tool that the agent can discover and use to fulfill user requests. It essentially registers this function within the agent's toolkit.

* **Docstring Notes**: The text within the triple quotes (`"""Docstring goes here"""`) serves as crucial instructions for the AI agent. It explains:
    * **What the tool does**: "Provides the user's location upon request."
    * **How it works**: Mentions using the `geocoder` library and determining location via IP address.
    * **Input parameters**: Describes the `query` parameter.
    * **Output**: Specifies what the tool returns (location information as a string).
    * **Example**: Shows how the tool can be used and its expected output.

The agent reads this docstring to understand the tool's purpose, how to call it (with what arguments), and what kind of result to expect. This allows the agent to strategically use the `get_location` tool when a user asks for their location.

> **Note:**
> This is just one way to create a custom tool, we can also import tools from libraries or use MCP Servers (Again, more on this later...) 
> 

In [6]:
from llama_stack_client.lib.agents.client_tool import client_tool

@client_tool
def get_location(query: str = "location"):
    """
    Provides the user's location upon request.

    This function uses the geocoder library to determine the user's location
    based on their IP address.  It returns the city, state, and country.

    :param query:  The query from the user.  Defaults to "location".
    :type query: str
    :return:  Information about the user's current location.
    :rtype: str

    Example:
        >>> get_location("where am i")
        "Your current location is: Some City, Some State, Some Country"
    """
    import geocoder
    try:
        g = geocoder.ip('me')
        if g.ok:
            return f"Your current location is: {g.city}, {g.state}, {g.country}" # can be modified to return latitude and longitude if needed
        else:
            return "Unable to determine your location"
    except Exception as e:
        return f"Error getting location: {str(e)}"

## Prompt Chaining (Optional)

Prompt chaining allows an AI agent to remember information across multiple turns within the same session. In this example, both user questions share the same `session_id`.

Because they share a session, the agent will remember the answer to the first question ("Where am I?") when interpreting the second question ("search for any weather-related risks in my area..."). This enables the agent to understand "my area" refers to the location identified in the previous turn, leading to a more context-aware and helpful response.

### Defining a Multi-tool Agent

This code initializes an AI agent with specific instructions and tools. 
The `tools` parameter is a list containing elements the agent can use. Here, it includes `get_location` (a custom function defined earlier) and `"builtin::websearch"` (a pre-built web searching capability in Llama Stack). 


The instructions (often refered to as "Prompt" guide the agent on when to use each of these tools based on the user's query.




In [7]:

agent = Agent(
    client, 
    model=SELECTED_MODEL,
    # instructions="""You are a helpful assistant. 
    # When a user asks about their location, you MUST use the get_location tool. When searching for nearby places, you MUST use the websearch tool.
    # """ ,
    instructions="""You are a helpful assistant. 
    When a user asks about their location, you MUST use the get_location tool. When searching for nearby places, you MUST use the brave_search tool.
    """ ,
    tools=[get_location, "builtin::websearch"],
    #sampling_params=sampling_params
)

### inspecting the Agent's response structure with Prompt Chaining

Notice that we are **NOT** generating a new session ID for each question, this way, the convesation memory is shared throughtout the session. 


In [10]:
user_prompts = [
    "Where am I?",
    "search for any weather-related risks in my area that could disrupt network connectivity or system availability?",
]

# Generate a new Unique Identifier for the session, but *NOT* for each question.

new_uuid = uuid.uuid4()
session_id = agent.create_session("prompt-chaining-session-{new_uuid}")

for prompt in user_prompts:
    print("\n"+"="*50)
    print(f"Processing user query: {prompt}")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )

    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: Where am I?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: get_location, Arguments: {'query': 'location'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


'Your current location is: Columbus, Ohio, US'


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
<|python_tag|>{"city": "Columbus", "state": "Ohio", "country": "US"}

========== Query processing completed ========== 


Processing user query: search for any weather-related risks in my area that could disrupt network connectivity or system availability?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'weather risks near Columbus, Ohio, US'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'weather risks near Columbus, Ohio, US',
│   'top_k': [
│   │   {
│   │   │   'title': 'Weather in Columbus, Ohio, US',
│   │   │   'url': 'https://www.weatherapi.com/',
│   │   │   'content': "{'location': {'name': 'Columbus', 'region': 'Ohio', 'country': 'United States of America', 'lat': 39.9611, 'lon': -82.9989, 'tz_id': 'America/New_York', 'localtime_epoch': 1747338421, 'localtime': '2025-05-15 15:47'}, 'current': {'last_updated_epoch': 1747338300, 'last_updated': '2025-05-15 15:45', 'temp_c': 27.8, 'temp_f': 82.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 9.4, 'wind_kph': 15.1, 'wind_degree': 196, 'wind_dir': 'SSW', 'pressure_mb': 1007.0, 'pressure_in': 29.74, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 55, 'cloud': 50, 'feelslike_c': 29.6, 'feelslike_f': 85.3, 'windchill_c': 28.1, 'windchill_f': 82.6, 'heatindex_c': 30.5, 'heatindex_f': 86.8, 'dewpoint_c': 20.9, 'dewpoint_f': 69.6, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 7.6, 'gust_mph': 10.8, 'gust_kph': 17.4}}",
│   │   │   'score': 0.8995267,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Columbus, Ohio weather forecast for May 15, 2025 - 10tv.com',
│   │   │   'url': 'https://www.10tv.com/video/weather/forecast/columbus-ohio-weather-forecast-for-may-15-2025-strong-storms-later-this-evening/530-6770c16e-36af-4f4a-a356-11f8042662c6',
│   │   │   'content': 'Thursday: Partly sunny, warm and a bit more humid. Isolated afternoon shower/storm. Few stronger storms Thursday night.',
│   │   │   'score': 0.8616433,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Damaging winds, hail, tornadoes possible ... - The Columbus Dispatch',
│   │   │   'url': 'https://www.dispatch.com/story/news/local/2025/05/15/columbus-weather-forecast-damaging-winds-hail-tornadoes-possible-thursday-friday-according-to-nws/83642723007/',
│   │   │   'content': 'Here is what you can expect from the weather for the next week. Ohio heat wave getting worse: New maps show forecast pushing 90. Where it will be hottest Columbus weather for the week of May 15, 2025',
│   │   │   'score': 0.83452857,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Weather in Columbus in May 2025 (Ohio) - Detailed Weather Forecast for ...',
│   │   │   'url': 'https://world-weather.info/forecast/usa/columbus_1/may-2025/',
│   │   │   'content': "Weather in Columbus in May 2025 (Ohio) - Detailed Weather Forecast for a Month Weather Weather in Columbus Weather in Columbus in May 2025 1 +79°+64° 2 +73°+63° 3 +52°+61° 4 +61°+50° 5 +63°+52° 6 +54°+52° 7 +70°+55° 8 +66°+59° 9 +61°+48° 10 +72°+46° 11 +77°+57° 12 +72°+63° 13 +72°+64° 14 +75°+63° 15 +81°+64° 16 +84°+73° 17 +75°+70° 18 +68°+63° 19 +66°+55° 20 +63°+46° 21 +64°+52° 22 +72°+57° 23 +68°+57° 24 +64°+55° 25 +72°+59° 26 +70°+63° 27 +75°+64° 28 +61°+64° 29 +79°+57° +79°+70° +81°+70° HourlyWeek10-Day14-Day30-DayYear Weather in Washington, D.C.+63° Hilliard+63° Lima+61° Mansfield+61° Marion+61° Newark+63° Sunbury+61° Tiffin+61° Findlay+61° Chillicothe+57° Parkersburg+59° Amelia+59° Circleville+61° Dayton+63° Gallipolis+61° Patriot+61° Berkshire+61° world's temperature today day day Weather forecast on your site Install Columbus+63° Temperature units",
│   │   │   'score': 0.7703443,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Columbus Weather: Severe storms possible in Central Ohio as heat and ...',
│   │   │   'url': 'https://abc6onyourside.com/news/local/columbus-weather-potential-for-storms-on-thursday-and-friday-alert-storms-rain-temperatures-central-ohio-reynoldsburg-dublin-gahanna-grove-city-polaris-groveport-obetz-worthington-upper-arlington-wx',
│   │   │   'content': 'More scattered severe storms are possible tomorrow and areas along and south of I-70 for Ohio may have the best chance for severe storms for Friday and Friday night.',
│   │   │   '


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Based on the search results, there is a risk of damaging winds, hail, and tornadoes possible in Columbus, Ohio, US on Thursday and Friday. It is recommended to check the latest weather forecast for the most up-to-date information.

========== Query processing completed ========== 



## Lab Summary: Interacting with Agents and Tools

In this lab, you took your first steps in understanding and interacting with AI agents using the Llama Stack framework and observe the benefits of Llama Stack in providing a structured and intuitive way to build intelligent agents with access to a variety of tools, both built-in and custom, and the ability to manage conversational memory for more coherent interactions. 

You learned how to:

* **Define a basic AI agent:** You created an agent and provided it with instructions on how to behave.
* **Utilize built-in tools:** You explored how agents can leverage pre-existing tools within Llama Stack, specifically the `websearch` tool, to gather information from the web.
* **Observe agent behavior:** You became familiar with the concepts of "Turns" and "Steps" that an agent goes through when processing a query, including the crucial "Tool Call" step where the agent decides to use a tool.
* **Create custom tools:** You experienced how to extend an agent's capabilities by defining your own specialized tool (`get_location`) using Python and decorators.
* **Implement prompt chaining:** You explored how to maintain context across multiple user interactions within the same session, allowing the agent to remember previous information and use it to answer subsequent questions more effectively.

Through these exercises, you gained a foundational understanding of how agents operate within the Llama Stack ecosystem and how they can be equipped with tools to perform specific tasks. 